In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from torchvision.transforms.v2 import MixUp

# Test-Time Augmentation (TTA) Function
def tta_predict(model, image, transforms_list, device):
    model.eval()
    with torch.no_grad():
        preds = [model(transform(image[0]).unsqueeze(0).to(device)) for transform in transforms_list]
    return torch.mean(torch.stack(preds), dim=0)

# TTA Transforms 
tta_transforms = [
    transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.ToTensor()]),
    transforms.Compose([transforms.RandomRotation(30), transforms.ToTensor()]),
    transforms.Compose([transforms.ColorJitter(brightness=0.2), transforms.ToTensor()])
]

# Define Training Transform (Now Properly Modified)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    AutoAugment(AutoAugmentPolicy.IMAGENET),  # AutoAugment added
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define Validation & Test Transforms (No TTA Here)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
#Load dataset with correct transforms
train_dataset = ImageFolder(root='path/to/your/data', transform=train_transform)
val_dataset = ImageFolder(root='path/to/your/data', transform=val_test_transform)
test_dataset = ImageFolder(root='path/to/your/data', transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from torch.optim.lr_scheduler import CyclicLR

# Define Custom Model
class CustomResNet(nn.Module):
    def __init__(self, base_model):
        super(CustomResNet, self).__init__()
        self.base = nn.Sequential(*list(base_model.children())[:-1])  # Remove final FC layer
        self.dropout = nn.Dropout(0.5)  # 50% Dropout to prevent overfitting
        self.fc = nn.Linear(base_model.fc.in_features, 2)

    def forward(self, x):
        x = self.base(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

# Load ResNet18 and apply custom classifier
base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model = CustomResNet(base_model)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-2, step_size_up=2000, mode="triangular2")

In [ ]:
# Get the number of classes dynamically
num_classes = len(train_dataset.classes)

# Apply MixUp with num_classes
torchvision_mixup = MixUp(alpha=0.2, num_classes=num_classes)
# Training Loop with Validation
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Convert labels to one-hot encoding
        labels = torch.nn.functional.one_hot(labels, num_classes=num_classes).float()
        
        # Apply MixUp
        images, labels = torchvision_mixup(images, labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)  # Using loss_fn with label smoothing
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Update learning rate
    scheduler.step()

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")


In [ ]:
# Test Set Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# Save trained model
torch.save(model.state_dict(), "path/to/your/model")

In [ ]:
model.load_state_dict(torch.load("path/to/your/model", map_location=device))

from PIL import Image
import torchvision.transforms as transforms

image_path = "path/to/your/test/image"
image = Image.open(image_path)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust based on your model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

input_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension

with torch.no_grad():
    output = model(input_tensor)

predicted_class = torch.argmax(output, dim=1).item()
print(f"Predicted Class: {predicted_class}")

import matplotlib.pyplot as plt

plt.imshow(image)
plt.title(f"Predicted: {predicted_class}")
plt.show()